In [1]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import nltk

# Download NLTK stopwords if not already present
nltk.download('stopwords')

# Function to clean the text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()  # Remove whitespaces
    # Optionally remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Load the dataset
file_path = 'data/reddit_jokes_slim_processed.csv'  # Update  your file pathwith
jokes_df = pd.read_csv(file_path)

jokes_df['combined_text'] = jokes_df['thread_title'] + " " + jokes_df['thread_selftext']
# Clean
jokes_df['cleaned_text'] = jokes_df['combined_text'].apply(clean_text)

# Bucketing the upvote ratios into 10 equal-sized classes using quantiles
jokes_df['upvote_class'] = pd.qcut(jokes_df['thread_upvote_ratio'], q=4, labels=False, duplicates='drop')

# Feature Engineering using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust the number of features
X_tfidf = tfidf_vectorizer.fit_transform(jokes_df['cleaned_text']).toarray()

# Encoding the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(jokes_df['upvote_class'])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# Training the Logistic Regression model
log_reg_model = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence
log_reg_model.fit(X_train, y_train)

# Predicting on the test set
y_pred_log_reg = log_reg_model.predict(X_test)

# Evaluating the model
evaluation_report_log_reg = classification_report(y_test, y_pred_log_reg)
print(evaluation_report_log_reg)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alishahed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

           0       0.34      0.39      0.36      2910
           1       0.25      0.22      0.23      2716
           2       0.29      0.28      0.29      3040
           3       0.34      0.35      0.34      2499

    accuracy                           0.31     11165
   macro avg       0.30      0.31      0.31     11165
weighted avg       0.30      0.31      0.30     11165



In [2]:
import pandas as pd
import re
import string
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import nltk

# Download NLTK stopwords if not already present
nltk.download('stopwords')

# Function to clean the text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()  # Remove whitespaces
    # Optionally remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Load the dataset
file_path = 'data/reddit_jokes_slim_processed.csv'  # Update with your file path
jokes_df = pd.read_csv(file_path)

# Clean the text in the dataset
jokes_df['cleaned_text'] = jokes_df['thread_selftext'].apply(clean_text)

# Bucketing the upvote ratios into 10 equal-sized classes using quantiles
jokes_df['upvote_class'] = pd.qcut(jokes_df['thread_upvote_ratio'], q=4, labels=False, duplicates='drop')

# Feature Engineering using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust the number of features
X_tfidf = tfidf_vectorizer.fit_transform(jokes_df['cleaned_text']).toarray()

# Encoding the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(jokes_df['upvote_class'])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# Training the LightGBM model
lgbm_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
lgbm_model.fit(X_train, y_train)

# Predicting on the test set
y_pred_lgbm = lgbm_model.predict(X_test)

# Evaluating the model
evaluation_report_lgbm = classification_report(y_test, y_pred_lgbm)
print(evaluation_report_lgbm)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alishahed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.048635 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 72998
[LightGBM] [Info] Number of data points in the train set: 26050, number of used features: 999
[LightGBM] [Info] Start training from score -1.353591
[LightGBM] [Info] Start training from score -1.403251
[LightGBM] [Info] Start training from score -1.323581
[LightGBM] [Info] Start training from score -1.470929
              precision    recall  f1-score   support

           0       0.31      0.54      0.39      2910
           1       0.24      0.16      0.20      2716
           2       0.30      0.23      0.26      3040
           3       0.36      0.28      0.32      2499

    accuracy                           0.30     11165
   macro avg       0.30      0.30      0.29     11165
weighted avg       0.30      0.30      0.29     1